1. Install needed nugets

In [1]:
#r "nuget:Microsoft.ML,*-*"
#r "nuget:Microsoft.ML.Recommender,*-*"
#r "nuget:Microsoft.Data.Analysis, *-*"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"

Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22313.1 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.22504.6 Microsoft.ML, 2.0.0-preview.22313.1 Microsoft.ML.Recommender, 0.20.0-preview.22313.1

2. Add global usings

In [1]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using System.IO;
using System.Text;

3. Add input data model

In [1]:
public class AnimeInput
{
	[LoadColumn(0)]
	public float UserId {get;set;}

	[LoadColumn(1)]
	public float AnimeId {get;set;}

	[LoadColumn(2)]
	public float Rating {get;set;}
}

4. Add output data model

In [1]:
public class AnimeOutput
{
	[ColumnName("Score")]
	public float Score {get;set;}
}

5. Load and split dataset to test and training sets

In [1]:
var mlContext = new MLContext();
var trainingData = mlContext.Data.LoadFromTextFile<AnimeInput>("../../../ML.NET.Demo/Assets/rating.csv", hasHeader: true, separatorChar: ',');
var dataSplit = mlContext.Data.TrainTestSplit(trainingData, testFraction: 0.2);

6. *Optional* Display dataset

In [1]:
dataSplit.TestSet.ToTabularDataResource.Display();

7. Train the model (Feature Engineering)

In [1]:
var model = mlContext.Recommendation().Trainers.MatrixFactorization(labelColumnName: "Label", matrixColumnIndexColumnName: "UserFeature", matrixRowIndexColumnName: "AnimeFeature", approximationRank: 5, learningRate: 0.01, numberOfIterations: 50);

var pipeline = mlContext.Transforms.Conversion.MapValueToKey("UserFeature", nameof(AnimeInput.UserId))
					.Append(mlContext.Transforms.Conversion.MapValueToKey("AnimeFeature", nameof(AnimeInput.AnimeId)))
					.Append(mlContext.Transforms.CopyColumns("Label", nameof(AnimeInput.Rating)))
					.Append(model);

var trainedModel = pipeline.Fit(dataSplit.TrainSet);

8. Evaluate Model

In [1]:
var testSet = trainedModel.Transform(dataSplit.TestSet);
var metrics = mlContext.Recommendation().Evaluate(testSet);

metrics

MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
1.424727638001567,4.813218505218332,2.1939048532737995,4.813218505394835,0.657061429675964


9. Save the model

In [1]:
mlContext.Model.Save(trainedModel, dataSplit.TrainSet.Schema, "./Recommendation.mdl");

10. Load the model and test

In [1]:
var newSample = new AnimeInput
{
	UserId = 1f,
	AnimeId = 20f
};

using (var stream = new FileStream("./Recommendation.mdl", FileMode.Open, FileAccess.Read))
{
    var model = mlContext.Model.Load(stream, out var _);
    var predictionEngine = mlContext.Model.CreatePredictionEngine<AnimeInput, AnimeOutput>(model);
    var results = predictionEngine.Predict(newSample);

    results.Display();
}

Score
-0.6629313
